# Visualizing uncertainty

Given a value $n$, how confident are we that it is contained in a set of observations?

This interactive plot allows the user to compare the height/confidence intervals of randomly generated sets of observations, by adapting the color of the bars to the chosen $n$. If $n$ is _very likely_ (that is, two standard deviations away from the mean) larger than our observations, the bar is red. If $n$ is _ver likely_ lower, the bar is blue. For values closer to the mean, the bar color is picked from a diverging Red-Blue colormap.

The horizontal line can be dragged to set $n$. 
 
Possible improvements/fixes:
 - Allow user to set a y-interval instead of just a y-value;
 - Inspect a bar by clicking on it: the other bars are colored depending on how likely the observations are to be higher/lower than in the selected bar;
 - Make this prettier;
 
_Inspired by the coursera "Applied data science with Python" course from the University of Michigan. This is Assignment 3 from the "Applied Plotting, Charting & Data Representation" module, with slight modifications._

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm  # colormaps
import matplotlib as mpl

In [2]:
%matplotlib widget

In [3]:
class DraggableLine:
    def __init__(self, line):
        self.lne = line
        self.press = None

    def connect(self):
        self.cidpress = self.lne.figure.canvas.mpl_connect(
            'button_press_event', self.on_press)
        self.cidrelease = self.lne.figure.canvas.mpl_connect(
            'button_release_event', self.on_release)
        self.cidmotion = self.lne.figure.canvas.mpl_connect(
            'motion_notify_event', self.on_motion)

    def on_press(self, event):
        'On button press, check whether mouse is on line'
        if event.inaxes != self.lne.axes: return
        contains, indices = self.lne.contains(event)
        if contains:
            self.lne.set_ydata(event.ydata)
            self.press = True

    def on_motion(self, event):
        'on motion, move the line'
        if self.press is None: return
        if event.inaxes != self.lne.axes: return
        self.lne.set_ydata(event.ydata)

        self.lne.figure.canvas.draw()



    def on_release(self, event):
        'on release we reset the press data'
        self.press = None
        self.lne.figure.canvas.draw()

    def disconnect(self):
        'disconnect all the stored connection ids'
        self.rect.figure.canvas.mpl_disconnect(self.cidpress)
        self.rect.figure.canvas.mpl_disconnect(self.cidrelease)
        self.rect.figure.canvas.mpl_disconnect(self.cidmotion)


class ColorChangingBar:
    def __init__(self, rect, err, cmap, draggableline):
        self.rect = rect
        self.height = self.rect.get_height()
        self.err = err
        self.press = None
        self.colormap = cmap
        self.lne = draggableline

    def connect(self):
        self.cidpress = self.rect.figure.canvas.mpl_connect(
            'button_press_event', self.on_press)
        self.cidrelease = self.rect.figure.canvas.mpl_connect(
            'button_release_event', self.on_release)
        self.cidmotion = self.rect.figure.canvas.mpl_connect(
            'motion_notify_event', self.on_motion)

    def change_colors(self, n):
        confidence = (n - self.height) / (self.err)
        self.rect.set(color=self.colormap.to_rgba(confidence))

    def on_press(self, event):
        'on button press, store some data'
        if event.inaxes != self.rect.axes: return
        contains, indices = self.lne.contains(event)
        if contains:
            self.press = True
            self.change_colors(int(event.ydata))

    def on_motion(self, event):
        if self.press is None: return
        if event.inaxes != self.rect.axes: return
        self.change_colors(int(event.ydata))

    def on_release(self, event):
        'on release we reset the press data'
        self.press = None

    def disconnect(self):
        'disconnect all the stored connection ids'
        self.rect.figure.canvas.mpl_disconnect(self.cidpress)
        self.rect.figure.canvas.mpl_disconnect(self.cidrelease)
        self.rect.figure.canvas.mpl_disconnect(self.cidmotion)


In [4]:
np.random.seed(12345)

df = pd.DataFrame([np.random.normal(32000,200000,3650),
                   np.random.normal(43000,100000,3650),
                   np.random.normal(43500,140000,3650),
                   np.random.normal(48000,70000,3650)],
                  index=[1992,1993,1994,1995])

df['mean'] = df.mean(axis = 1)
df['std'] = 2 * df.sem(axis = 1)

In [5]:
fig, ax = plt.subplots()

norm = mpl.colors.Normalize(vmin=-1, vmax=1)
my_colors = cm.ScalarMappable(cmap='RdBu', norm=norm)
plt.colorbar(my_colors)

ax.set(facecolor='white')
for spine in ['top', 'bottom', 'left', 'right']:
    ax.spines[spine].set_visible(False)

ax.xaxis.set(ticks=range(1992, 1996))
bars = ax.bar(df.index, df['mean'], yerr=df['std'], color='tab:gray', capsize=5)

hline = ax.axhline(y=20000, color='black', lw=1.1, ls="--")
dl = DraggableLine(hline)
dl.connect()

mybars = []
for bar, err in zip(bars, df['std']):
    colbar = ColorChangingBar(bar, err, my_colors, hline)
    colbar.connect()
    mybars.append(colbar)  # reference so callback doesn't get garbaged 
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …